In [85]:
import pandas as pd
import numpy as np

# 1. to define the datasets
to divide it into train and test sets.

In [86]:
# to define the datasets

dataset = pd.read_csv('watermelon3.0a.csv', sep=',')

dataset = np.array(dataset)

# print('before')
# print(dataset)

# translate '是' and '否' into 1 and 0
for i in range(len(dataset)):
    dataset[i][3] = 1 if dataset[i][3] == '是' else 0


dataset = dataset.T
dataset = dataset[1:4]
# print('after altering')
print(dataset)

print(dataset.shape)

d = int(17 * 0.7)

# [0:2, 0:d] [2:3, 0:d] [0:2, d:] [2:3, d:]
train_x = dataset[0:2, 0:d]
train_y = dataset[2:3, 0:d]
test_x = dataset[0:2, d:]
test_y = dataset[2:3, d:]

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

train_plus = np.ones(d).reshape(1, -1)
test_plus = np.ones(17 - d).reshape(1, -1)

print('train_plus shape', train_plus.shape)

train_x = np.append(train_x, train_plus, axis=0)
test_x = np.append(test_x, test_plus, axis=0)

print('train_x is ', train_x)
print('train_x shape is', train_x.shape)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

[[0.697 0.774 0.634 0.608 0.556 0.403 0.481 0.437 0.666 0.243 0.245 0.343
  0.639 0.657 0.36 0.593 0.719]
 [0.46 0.376 0.264 0.318 0.215 0.237 0.149 0.211 0.091 0.267 0.057 0.099
  0.161 0.198 0.37 0.042 0.103]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]]
(3, 17)
(2, 11) (1, 11) (2, 6) (1, 6)
train_plus shape (1, 11)
train_x is  [[0.697 0.774 0.634 0.608 0.556 0.403 0.481 0.437 0.666 0.243 0.245]
 [0.46 0.376 0.264 0.318 0.215 0.237 0.149 0.211 0.091 0.267 0.057]
 [1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0]]
train_x shape is (3, 11)
(3, 11) (1, 11) (3, 6) (1, 6)


# 2. initialize the parameters

In [96]:
# initialize the parameters

def initialize_with_zeros(dim : int):
    return np.random.randn(dim, 1)

initialize_with_zeros(3)

numpy.ndarray

# 3. propagate

In [88]:
m = 10
Beta = np.random.randn(3, 1)
X = np.random.randn(3, m)
Y = np.random.randn(1, m)

temp = -1 * Y * np.dot(Beta.T, X) + np.log(1 + np.exp(np.dot(Beta.T, X)))
print(temp.shape)

cost = np.sum(-1 * Y * np.dot(Beta.T, X) + np.log(1 + np.exp(np.dot(Beta.T, X))))

print(cost)

(1, 10)
35.73988578254155


In [89]:
def propagate(Beta:np.ndarray, X:np.ndarray, Y:np.ndarray):
    cost = np.sum(-1 * Y * np.dot(Beta.T, X) + np.log(1.0 + np.exp(np.dot(Beta.T, X).astype(np.ndarray))))
    p1 = np.exp(np.dot(Beta.T, X)) / (1 + np.exp(np.dot(Beta.T, X)))
    db = -1 * np.sum(X * (Y - p1))
    ddb = np.sum(np.dot(X.T, X) * p1 * (1 - p1))

    grads = {
        'db': db,
        'ddb': ddb
    }

    return grads, cost

# 4. Optimize

In [90]:
def optimize(Beta:np.ndarray, X:np.ndarray, Y:np.ndarray, num_iterations=1000):
    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(Beta, X, Y)

        db = grads['db']
        ddb = grads['ddb']

        Beta = Beta - ddb ** (-1) * db
        
        if i % 100 == 0:
            costs.append(cost)
        
        if i % 100 == 0:
            print('Now cost is ', cost)

    parameters = {
        'Beta': Beta,
        'costs': costs
    }

    return parameters

# 5. Prediction

In [91]:
def predict(Beta:np.ndarray, X)->np.ndarray:

    m = len(X)
    y_prediction = np.zeros((len(m), 1))

    Y = np.dot(Beta.T, X)

    for i in range(m):
        if Y[i] >= 0.5:
            y_prediction[i] = 1
    
    return y_prediction

# 6. Model construction

In [92]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([2, 3, 4, 4, 5])

c = np.sum(a == b)

print(c)

2


In [93]:
def model(train_x:np.ndarray, train_y:np.ndarray, test_x:np.ndarray, test_y:np.ndarray, num_iterations=1000):
    dim = 3
    Beta = initialize_with_zeros(dim)
    parameters = optimize(Beta, train_x, train_y, num_iterations)

    Beta = parameters['Beta']
    costs = parameters['costs']

    y_prediction_train = predict(Beta, train_x)
    y_prediction_test = predict(Beta, test_x)

    correct_rate_train = np.sum(y_prediction_train == train_y)
    correct_rate_test = np.sum(y_prediction_test, test_y)

    print('correct_rate_train is {} %', correct_rate_train / 100)
    print('correct_rate_test is {} %', correct_rate_test / 100)

    d = {
        'y_prediction_train': y_prediction_train,
        'y_prediction_test': y_prediction_test,
        'Beta': Beta, 
        'costs': costs
    }
    
    return d

# 7. Over, But is it ok?

In [94]:
logistic_regression = model(train_x, train_y, test_x, test_y)


TypeError: loop of ufunc does not support argument 0 of type float which has no callable exp method